## Paso 1 -  Modelo base (CNN)

En este notebook se desarrolla el punto 1 del caso práctico:

**"Modelo base basado en redes neuronales convolucionales"**

La arquitectura de la red es libre, pero debe incluir como mínimo:
- una **capa convolucional**,
- una **capa de pooling** (el tipo es libre),
- seguida de una **capa densa de salida**.

> En el siguiente apartado se implementará un modelo más complejo.

---

### Arquitectura del modelo

El modelo desarrollado está compuesto por las siguientes capas:

1. **Capa de entrada (`Input`)**  
   Define la forma de entrada de las imágenes: `(150, 150, 3)`, correspondiente a imágenes RGB de 150x150 píxeles.

2. **Capa convolucional (`Conv2D`)**  
   Aplica 32 filtros de tamaño 3x3 para extraer características espaciales de las imágenes.  
   Utiliza la función de activación `ReLU` para introducir no linealidad.

3. **Capa de agrupamiento (`MaxPooling2D`)**  
   Reduce la dimensión espacial seleccionando el valor máximo en una ventana de 2x2.  
   Ayuda a disminuir el número de parámetros y a prevenir el sobreajuste.

4. **Capa de aplanamiento (`Flatten`)**  
   Convierte las características extraídas en un vector unidimensional para conectarlo con las capas densas.

5. **Capa densa intermedia (`Dense`)**  
   Contiene 64 neuronas con activación `ReLU`. Aprende patrones complejos de alto nivel.

6. **Capa de salida (`Dense`)**  
   Tiene 6 neuronas (una por clase del dataset), con activación `softmax` para clasificación multiclase.

---

### Guardado del modelo

Una vez entrenado, el modelo y su historial de entrenamiento se guardan en la carpeta `models`.

---

## Nota importante: estructura modular del proyecto

Dado que todos los notebooks comparten las funciones de lectura de datos y guardado de modelos, se han creado dos módulos en la carpeta `utilities`:

---

### Módulo `dataloader.py`

#### ⚙️ Utilidades para la gestión de datos: Generadores personalizados y carga desde `.npy`

Este módulo define las herramientas necesarias para cargar y preparar los datos de imagen para su uso en modelos de clasificación con TensorFlow/Keras. Está diseñado para funcionar con arrays NumPy previamente generados (`images.npy` y `categories.npy`), optimizando el uso de memoria y permitiendo configuraciones personalizadas en tiempo de entrenamiento.

##### Componentes principales

- **`DataGenerator`**: clase derivada de `tf.keras.utils.Sequence` que permite iterar sobre lotes de imágenes en memoria.
  - Admite redimensionado automático (`resize_to=(h, w)`).
  - Permite normalización de los píxeles (`normalize=True`).
  - Implementa barajado de datos al final de cada época.

- **`PreprocessedDataGenerator`**: extiende `DataGenerator` para aplicar funciones de preprocesamiento personalizadas (por ejemplo, `preprocess_input` de MobileNet, ResNet, etc.).

- **`load_data_npy()`**: función para cargar datos desde archivos NumPy (`images.npy`, `categories.npy`) y dividirlos en:
  - Conjunto de entrenamiento (60%)
  - Validación (15%)
  - Test (25%)
  
  Las divisiones siguen las proporciones especificadas en el enunciado de la práctica, y permiten trabajar de forma flexible con distintos generadores y modelos.

Este módulo está pensado para facilitar la reutilización de datos procesados y entrenar modelos de forma eficiente, tanto en CPU como en GPU.

### Módulo` dataloader.py`

Este módulo proporciona funciones auxiliares para guardar y cargar modelos entrenados con Keras junto con sus historiales de entrenamiento (`history`). Facilita la gestión de resultados experimentales y permite recuperar modelos guardados para evaluación o reutilización posterior.

### Funciones disponibles

- **`save_model_and_history(model, history, model_path='model', history_path=None)`**  
  Guarda el modelo en formato `.h5` y el historial de entrenamiento como archivo `.json`.  
  - Por defecto, los archivos se almacenan en una carpeta `models/` en el directorio raíz del proyecto.
  - El nombre del historial se genera automáticamente a partir del nombre del modelo, a menos que se indique explícitamente.

- **`load_model_and_history(model_name='model')`**  
  Carga un modelo `.h5` y su historial asociado `.json` desde la carpeta `models/`.  
  Devuelve una tupla `(modelo, history)` lista para usar en evaluación o visualización.

- **`load_history(history_path)`**  
  Carga directamente un historial de entrenamiento desde un archivo `.json`.

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from utils.dataloader import load_data_npy, DataGenerator
from utils.model_utils import save_model_and_history

2025-05-27 13:45:25.249857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 13:45:25.730969: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 13:45:26.073045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748353526.426095   48528 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748353526.503249   48528 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748353527.285870   48528 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Cargar los datos
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

# Crear generadores
train_gen = DataGenerator(images_train, categories_train)
val_gen = DataGenerator(images_val, categories_val, shuffle=False)
test_gen = DataGenerator(images_test, categories_test, shuffle=False)

print(f"Train: {images_train.shape}, Validation: {images_val.shape}, Test: {images_test.shape}")

Train: (10220, 150, 150, 3), Validation: (2555, 150, 150, 3), Test: (4259, 150, 150, 3)


In [4]:
# Arquitectura CNN base
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=30,
                    callbacks=[early_stop])

# Evaluación en test
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}  |  Test Loss: {test_loss:.4f}")

# Guardar modelo e historial
save_model_and_history(
    model,
    history,
    model_path='model_base_cnn'
)

Epoch 1/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 127s 390ms/step - accuracy: 0.4798 - loss: 2.0923 - val_accuracy: 0.6708 - val_loss: 0.8972
Epoch 2/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 150s 469ms/step - accuracy: 0.7757 - loss: 0.6521 - val_accuracy: 0.7292 - val_loss: 0.7717
Epoch 3/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 121s 379ms/step - accuracy: 0.8781 - loss: 0.3695 - val_accuracy: 0.7503 - val_loss: 0.7747
Epoch 4/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 120s 375ms/step - accuracy: 0.9491 - loss: 0.1814 - val_accuracy: 0.7358 - val_loss: 0.9345
Epoch 5/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 124s 386ms/step - accuracy: 0.9745 - loss: 0.1076 - val_accuracy: 0.7229 - val_loss: 1.0237
Epoch 6/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 173s 541ms/step - accuracy: 0.9794 - loss: 0.0782 - val_accuracy: 0.7335 - val_loss: 1.1910
Epoch 7/30
320/320 ━━━━━━━━━━━━━━━━━━━━ 159s 498ms/step - accuracy: 0.9939 - loss: 0.0331 - val_accuracy: 0.7354 - val_loss: 1.2245
134/134 ━━━━━━━━━━━━━━━━━━━━ 17s 127ms/step - accuracy: 0.7399 - loss: 0.763


Test Accuracy: 0.7387  |  Test Loss: 0.7632
Modelo guardado en: /opt/notebooks/M9/models/model_base_cnn.h5
Historial guardado en: /opt/notebooks/M9/models/model_base_cnn_history.json
